# Sentiment Analysis by Pre-developed Twitter Library -- Sentiment Intensity Analyzer (SIA)


#### By Joyce Jiang | Code by Joyce¶

### Citation & Source of my code

Majority of my code is inspired by ayushs136 's tutorial Jupyter Notebook from Github, you can check it out at https://github.com/llSourcell/Sentiment_Analysis/blob/master/Sentiment_Analysis.ipynb

In [1]:
import nltk
import re
import string
import pandas as pd
from pandas import DataFrame
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [2]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Joyce\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [3]:
df= pd.read_excel('Labeled/Covid19UG_NoDuplicates23_05_20_labeled.xlsx',index=False)

df

,Unnamed: 0,Row ID,Tweet ID,Username,Tweet,Time,Tweet Type,Retweeted By,Number of Retweets,Hashtags,Mentions,Name,Location,Web,Bio,Number of Tweets,Number of Followers,Number Following,Location Coordinates,SENTIMENT
0,0,1,1263678902041526016,AhmadiyyaUg,His excellency president #Uganda @KagutaMuseve...,5/21/2020 8:51:42 PM,Retweet,AhmadiyyaKings1,4,Uganda Ahmadiyya COVID19UG,KagutaMuseveni HumanityFirstUK UgHumanityfirst...,AhmadiyyaUganda,NaN,NaN,Official Twitter Handle of Uganda Ahmadiyya Mu...,110,155,438,NaN,1.0
1,1,4,1263678714124153088,NansinguzaJ,@Katabasasa @nbstv #NBSFrontline #NBSUpdates\n...,5/21/2020 8:50:58 PM,Tweet,NaN,0,NBSFrontline NBSUpdates COVID19UG Covid_19,Katabasasa nbstv,Nansinguza Jacob,Uganda,https://t.co/8zcFVolmZH,#VisualCommunication #MedicaIillustration & #E...,38917,3300,4103,+01.25+032.5/,NaN
2,2,7,1263677791222074880,SolomonMGrace2,Mr. ERIAS Lukwago should know that SOPs are in...,5/21/2020 8:47:18 PM,Retweet,kugonza6,1,NaN,JudiciaryUG peter_katwesige OfwonoOpondo,Male Solomon Grace,"Kampala, Uganda",https://t.co/CUY7tvkw87,"Public Administrator, Policy Researcher and An...",8851,1246,5009,+00.31628+032.58219/,NaN
3,3,8,1263676429994254080,Cotildakhainza,Energy Minister @DrKitutu hands over the Ugx.2...,5/21/2020 8:41:53 PM,Retweet,h_ssali,18,COVID19UG,DrKitutu Parliament_Ug,Psycho,"Kampala, Uganda",NaN,Student of MAKERERE University Kampala. PURSUI...,1636,1341,1206,+00.31628+032.58219/,NaN
4,4,9,1263674084405252096,matookerepublic,15 new COVID-19 cases have been confirmed by t...,5/21/2020 8:32:34 PM,Retweet,KKabajwisa,1,NaN,NaN,Matooke Republic,"Kampala, Uganda",https://t.co/4au2z8o6u8,Freshly Peeled Info,7186,2516,342,+00.31628+032.58219/,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5056,5056,16824,1260636518072421888,OWC_ug,@OworiSylvia : We have been engaging directly ...,5/13/2020 11:22:22 AM,Retweet,jbogube,2,NaN,OworiSylvia OPMUganda,Wealth Creation Ug,Uganda,https://t.co/QacjAzznSK,Our mission is ''To improve household income t...,2328,3353,715,+01.25+032.5/,NaN
5057,5057,16826,1260636408328356096,OworiSylvia,Major Rubalamira: @OWC_ug has been supplied by...,5/13/2020 11:21:55 AM,Retweet,jbogube,14,NaN,OWC_ug GovUganda OPMUganda MAAIF_Uganda Parlia...,Sylvia Damalie Owori,NaN,NaN,Exec Assistant to the CC of Operation Wealth C...,48,83,107,NaN,NaN
5058,5058,16838,1260630805971374080,i_kagoya,It is going to be very difficult for governmen...,5/13/2020 10:59:40 AM,Retweet,ElliotOrizaarwa,1,NaN,KagutaMuseveni RuhakanaR,Irene Kagoya,Kampala Uganda,NaN,"A lawyer, & Women's Rights Activist, workin...",3826,792,851,+00.31628+032.58219/,NaN
5059,5059,16839,1260630418816225024,JenifaOchwo,For making the #COVID19UG lockdown bearable......,5/13/2020 10:58:07 AM,Retweet,TheLoveDre,1,COVID19UG ThankYouUgandans,NaN,Jenifa Ochwo,Uganda,NaN,Thinker | Faith | Entertainment | Justice | Go...,30887,3221,831,+01.25+032.5/,NaN


In [4]:
df=df[['Tweet','SENTIMENT']]

#### Capturing and defining positive and negative emoji

In [5]:
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])

emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])

emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)


emoticons = emoticons_happy.union(emoticons_sad)

#### Cleaning text to feed sentiment analytic tool SIA

In [6]:
def clean_tweets(tweet):
 
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(tweet)
#after tweepy preprocessing the colon symbol left remain after      #removing mentions
    tweet = re.sub(r':', '', tweet)
    tweet = re.sub(r'‚Ä¶', '', tweet)
#replace consecutive non-ASCII characters with a space
    tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)
#remove emojis from tweet
    tweet = emoji_pattern.sub(r'', tweet)
#filter using NLTK library append it to a string
    filtered_tweet = [w for w in word_tokens if not w in stop_words]
    filtered_tweet = []
#looping through conditions
    for w in word_tokens:
#check tokens against stop words , emoticons and punctuations
        if w not in stop_words and w not in emoticons and w not in string.punctuation:
            filtered_tweet.append(w)
    return ' '.join(filtered_tweet)
    #print(word_tokens)
    #print(filtered_sentence)return tweet

#### Loop through all tweets in [Tweet] column and apply SIA function on each of the tweet

In [7]:
sid = SentimentIntensityAnalyzer()
listy = []

for tweet in df['Tweet']:
    #re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", tweet)
    tweet= re.sub(r"http\S+", "", tweet)
    clean_text = clean_tweets(tweet)
    #print(tweet)
    ss = sid.polarity_scores(tweet)
    listy.append(ss)
    
#for tweet in df['Tweet']:
#    tweet= re.sub(r"http\S+", "", tweet)
#    clean_text = clean_tweets(tweet)

    
se = pd.Series(listy) 
df['polarity'] = se.values

df

C:\Users\Joyce\anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Tweet,SENTIMENT,polarity
0,His excellency president #Uganda @KagutaMuseve...,1.0,"{'neg': 0.0, 'neu': 0.753, 'pos': 0.247, 'comp..."
1,@Katabasasa @nbstv #NBSFrontline #NBSUpdates\n...,NaN,"{'neg': 0.0, 'neu': 0.915, 'pos': 0.085, 'comp..."
2,Mr. ERIAS Lukwago should know that SOPs are in...,NaN,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
3,Energy Minister @DrKitutu hands over the Ugx.2...,NaN,"{'neg': 0.0, 'neu': 0.902, 'pos': 0.098, 'comp..."
4,15 new COVID-19 cases have been confirmed by t...,NaN,"{'neg': 0.0, 'neu': 0.969, 'pos': 0.031, 'comp..."
...,...,...,...
5056,@OworiSylvia : We have been engaging directly ...,NaN,"{'neg': 0.0, 'neu': 0.917, 'pos': 0.083, 'comp..."
5057,Major Rubalamira: @OWC_ug has been supplied by...,NaN,"{'neg': 0.1, 'neu': 0.9, 'pos': 0.0, 'compound..."
5058,It is going to be very difficult for governmen...,NaN,"{'neg': 0.164, 'neu': 0.836, 'pos': 0.0, 'comp..."
5059,For making the #COVID19UG lockdown bearable......,NaN,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."


In [8]:
#Check details of [polarity]
df.polarity[0]

{'neg': 0.0, 'neu': 0.753, 'pos': 0.247, 'compound': 0.8588}

#### Loop through the dictionary in each row of [polarity], use append list function to append negative, positive, neutral, and compound values to each tweet.

In [9]:
Negative=[]
Neutral=[]
Positive=[]
Compound=[]
for x in range(len(df['Tweet'])):
    Negative.append(df.polarity[x]['neg'])
    Neutral.append(df.polarity[x]['neu'])
    Positive.append(df.polarity[x]['pos'])
    Compound.append(df.polarity[x]['compound'])

In [10]:
#result=df[['Negative','Netural','Positive','Compound']]
#df1=df[['Tweet','Sentiment','Questionable','Notes']]
df1=df[['Tweet','SENTIMENT']]

se1 = pd.Series(Negative)
df1['Negative'] = se1.values
se2 = pd.Series(Neutral)
df1['Neutral'] = se2.values
se3 = pd.Series(Positive)
df1['Positive'] = se3.values
se4 = pd.Series(Compound)
df1['Compound'] = se4.values
df1

,Tweet,SENTIMENT,Negative,Neutral,Positive,Compound
0,His excellency president #Uganda @KagutaMuseve...,1.0,0.000,0.753,0.247,0.8588
1,@Katabasasa @nbstv #NBSFrontline #NBSUpdates\n...,NaN,0.000,0.915,0.085,0.4215
2,Mr. ERIAS Lukwago should know that SOPs are in...,NaN,0.000,1.000,0.000,0.0000
3,Energy Minister @DrKitutu hands over the Ugx.2...,NaN,0.000,0.902,0.098,0.4588
4,15 new COVID-19 cases have been confirmed by t...,NaN,0.000,0.969,0.031,0.0772
...,...,...,...,...,...,...
5056,@OworiSylvia : We have been engaging directly ...,NaN,0.000,0.917,0.083,0.3818
5057,Major Rubalamira: @OWC_ug has been supplied by...,NaN,0.100,0.900,0.000,-0.4767
5058,It is going to be very difficult for governmen...,NaN,0.164,0.836,0.000,-0.8257
5059,For making the #COVID19UG lockdown bearable......,NaN,0.000,1.000,0.000,0.0000


In [11]:
#Convert [Polarity] to positive (1), negative (-1), and neutral (0) for comparison

prediction = [-1 if v < 0 else 1 if v>0 else 0 for v in df1['Compound']]
df1['Prediction']=prediction
df1

,Tweet,SENTIMENT,Negative,Neutral,Positive,Compound,Prediction
0,His excellency president #Uganda @KagutaMuseve...,1.0,0.000,0.753,0.247,0.8588,1
1,@Katabasasa @nbstv #NBSFrontline #NBSUpdates\n...,NaN,0.000,0.915,0.085,0.4215,1
2,Mr. ERIAS Lukwago should know that SOPs are in...,NaN,0.000,1.000,0.000,0.0000,0
3,Energy Minister @DrKitutu hands over the Ugx.2...,NaN,0.000,0.902,0.098,0.4588,1
4,15 new COVID-19 cases have been confirmed by t...,NaN,0.000,0.969,0.031,0.0772,1
...,...,...,...,...,...,...,...
5056,@OworiSylvia : We have been engaging directly ...,NaN,0.000,0.917,0.083,0.3818,1
5057,Major Rubalamira: @OWC_ug has been supplied by...,NaN,0.100,0.900,0.000,-0.4767,-1
5058,It is going to be very difficult for governmen...,NaN,0.164,0.836,0.000,-0.8257,-1
5059,For making the #COVID19UG lockdown bearable......,NaN,0.000,1.000,0.000,0.0000,0


In [12]:
#Drop the one with missing values of [SENTIMENT]

df_output=df1.dropna(subset=['SENTIMENT'])
df_output=df_output.reset_index(drop=True) #necessary to reset index: for the loop function to work
df_output

,Tweet,SENTIMENT,Negative,Neutral,Positive,Compound,Prediction
0,His excellency president #Uganda @KagutaMuseve...,1.0,0.000,0.753,0.247,0.8588,1
1,"Yes, he is one of the most celebrated Journali...",1.0,0.000,0.725,0.275,0.9493,1
2,"@aajtak Social distancing is a myth, anybody c...",-1.0,0.084,0.916,0.000,-0.5256,-1
3,#COVID19UG cases have been 'smalolized' from 2...,1.0,0.000,1.000,0.000,0.0000,0
4,#Football256 Update\n\n@ProlineFC are unconten...,-1.0,0.000,0.885,0.115,0.3818,1
...,...,...,...,...,...,...,...
66,"#COVID19UG WhatsApp chat both for accurate, tr...",1.0,0.000,0.790,0.210,0.8519,1
67,The day more than 317 Members of Parliament we...,-1.0,0.104,0.826,0.070,-0.1531,-1
68,"Timely, accurate information plays a critical ...",1.0,0.193,0.726,0.081,-0.4186,-1
69,Hello @HonAniteEvelyn it's really interseting ...,-1.0,0.080,0.920,0.000,-0.5661,-1


In [13]:
#Check if positive and negative comments are balanced in the [SENTIMENT] column
df.SENTIMENT.value_counts()

 1.0    36
-1.0    35
Name: SENTIMENT, dtype: int64

### Computing Accuracy (Overall, Negative, Positive)

In [14]:
n=0

for i in range(len(df_output['SENTIMENT'])):
    if df_output['SENTIMENT'][i]==df_output['Prediction'][i]:
        n+=1
OverallAccuracy = n/len(df_output['SENTIMENT'])*100

In [15]:
positive = df_output[df_output['SENTIMENT']==1]
negative = df_output[df_output['SENTIMENT']==-1]

positive=positive.reset_index(drop=True)
negative=negative.reset_index(drop=True)

ng=0

for i in range(len(negative['SENTIMENT'])):
    if negative['SENTIMENT'][i]==negative['Prediction'][i]:
        ng+=1
NegativeAccuracy = ng/len(negative['SENTIMENT'])*100

p=0

for i in range(len(positive['SENTIMENT'])):
    if positive['SENTIMENT'][i]==positive['Prediction'][i]:
        p+=1
PositiveAccuracy = p/len(positive['SENTIMENT'])*100


In [16]:
print("Overall Accuracy: "+ str(OverallAccuracy))
print("Negative Accuracy: "+ str(NegativeAccuracy))
print("Positive Accuracy: "+ str(PositiveAccuracy))

Overall Accuracy: 59.154929577464785
Negative Accuracy: 45.714285714285715
Positive Accuracy: 72.22222222222221


### Illustrate all labeled data for comparison

In [17]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', None)

#pd.reset('all') #to reset formatting

df_output

,Tweet,SENTIMENT,Negative,Neutral,Positive,Compound,Prediction
0,His excellency president #Uganda @KagutaMuseveni recognising efforts of #Ahmadiyya Muslim Community towards #COVID19UG. So grateful to our partners @HumanityFirstUK @UgHumanityfirst @HFI1995 4 support accorded to the ppl of Uganda. @ntvuganda #EidMubarak #EidUlFitr #NBSUpdates https://t.co/CLDFyJrLgx,1.0,0.000,0.753,0.247,0.8588,1
1,"Yes, he is one of the most celebrated Journalists of our time. \nYes, he is the reigning @BBC Komla Dumor award winner.\nBut, he is still humble enough to carry video equipment with the rest of us to the frontlines of the #COVID19 field.\nThe lesson is Humility.\n#COVID19UG https://t.co/RlZ8fxG6oh",1.0,0.000,0.725,0.275,0.9493,1
2,"@aajtak Social distancing is a myth, anybody can see this now a days, on road, in streets, at local market and at shops, it's very dangerous at this time #Social_Distancing #COVID19UG #aamaadmiparty #delhicm #ndtv #indiatv #hindustantimmes #timesofindia #bbcnews #ZeeNews #dainikbhasker",-1.0,0.084,0.916,0.000,-0.5256,-1
3,#COVID19UG cases have been 'smalolized' from 264 to 145 . \n\nWhere is the Luwero Hajji we celebrate?!,1.0,0.000,1.000,0.000,0.0000,0
4,#Football256 Update\n\n@ProlineFC are uncontended though they accept to the fact that they've been relegated due to the coronavirus pandemic \n\nhttps://t.co/3luQE42X6e #COVID19UG https://t.co/fZoKnervck,-1.0,0.000,0.885,0.115,0.3818,1
5,#COVID19UG messed me up... I would be here talking to my dog. https://t.co/1pUwbHpdqz,-1.0,0.194,0.806,0.000,-0.3400,-1
6,I don't think that #COVID19UG should stop elections. I don't think that the millions who vote attend political rallies #KatikkiroOnTheSPot #NTVOnTheSpot,-1.0,0.109,0.891,0.000,-0.2960,-1
7,"#nbsfrontline should change the opposition frontliners. None of them is discussing substance. All of them think President M7 has failed to manage #COVID19UG & are blaming his tea taking skills. If indeed he has failed, why do we have the least cases in the region &amp; no deaths?",-1.0,0.218,0.782,0.000,-0.9001,-1
8,For accurate info about #COVID19UG\n@modoanita\n https://t.co/WiD5p6i1GC,1.0,0.000,1.000,0.000,0.0000,0
9,@GodberTumushabe @JaneRuth_Aceng Nurses in Soroti also protesting...No clarity on what the Task Force funds are really doing...#COVID19UG,-1.0,0.000,0.856,0.144,0.4019,1


In [18]:
df1.to_excel('Output/Covid19UG_SentimentIntensityAnalyzer.xlsx')

In [19]:
df_output.to_excel('Output/Covid19UG_SentimentIntensityAnalyzer_59.15%_N45.71%_P72.22%.xlsx')